[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/vicente-gonzalez-ruiz/vector_quantization/blob/main/docs/gray_VQ.ipynb)

[![Colab](https://badgen.net/badge/Launch/on%20Google%20Colab/blue?icon=notebook)](https://colab.research.google.com/github/vicente-gonzalez-ruiz/vector_quantization/blob/main/docs/gray_VQ.ipynb)


# Vector Quantization (in the 2D domain) of a gray-scale image

We compute spatial VQ using [K-means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans).

In [ ]:
try:
    import matplotlib.pyplot as plt
except:
    !pip install matplotlib
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.axes as ax
    #plt.rcParams['text.usetex'] = True
    #plt.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}'] #for \text command
%matplotlib inline

In [ ]:
try:
    from sklearn.cluster import KMeans
    from sklearn.utils import shuffle
except:
    !pip install scikit-learn
    from sklearn.cluster import KMeans
    from sklearn.utils import shuffle

In [ ]:
try:
    from skimage import io
    from skimage.color import rgb2gray
except:
    !pip install scikit-image
    from skimage import io
    from skimage.color import rgb2gray

In [ ]:
try:
    import numpy as np
except:
    !pip install numpy
    import numpy as np

In [ ]:
try:
    import pylab
except:
    !pip install pylab
    import pylab

In [ ]:
import math
import os

In [ ]:
try:
    from information_theory.distortion import RMSE
except:
    !pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
    from information_theory.distortion import RMSE

## Configuration

In [ ]:
fn = "http://www.hpca.ual.es/~vruiz/images/lena.png"
block_width = 16
block_height = 16
n_clusters = 256  # Number of bins
N_tries = 3  # Number of times K-means is run
#range_of_N_bins = range(2, 256, 1)
range_of_N_bins = [1<<i for i in range(1, 8)]

## Read the image and show it

In [ ]:
#img = gray_image.read(fn, 0)
#gray_image.show(img, fn + "000.png")
img = io.imread(fn)
img = (rgb2gray(img)*256).astype(np.uint8)
plt.figure()
plt.title(fn)
io.imshow(img)
plt.show()

## Example

In [ ]:
block_length = block_width*block_height
np.random.seed(seed=1)  # makes the random numbers predictable
k_means = KMeans(init="k-means++", n_clusters=n_clusters, n_init=N_tries)
blocks = []
for i in range(0, img.shape[0], block_width):
    for j in range(0, img.shape[1], block_height):
        blocks.append(np.reshape(img[i:i + block_width, j:j + block_height], block_length))
blocks = np.asarray(blocks).astype(float)
k_means.fit(blocks)
centroids = k_means.cluster_centers_.squeeze().astype(np.uint8)  # the code-book
labels = k_means.labels_  # Labels of the centroids

labels = labels.reshape(img.shape[0]//block_height, img.shape[1]//block_width)
img_dequantized = np.empty_like(img)
for i in range(0, img.shape[0], block_width):
    for j in range(0, img.shape[1], block_height):
        img_dequantized[i:i + block_width, j:j + block_height] = centroids[labels[i//block_width,j//block_height]].reshape(block_height, block_width)

In [ ]:
#gray_image.show(img_dequantized, "Dequantized Image")
plt.figure()
plt.title("Dequantized Image")
io.imshow(img_dequantized)
plt.show()
assert len(centroids) == n_clusters
print("centroids =\n", centroids)
bits_per_block = int(math.log(n_clusters)/math.log(2))
print(f"{len(centroids)} centroids ({bits_per_block} bits/block)")
blocks_in_y = img.shape[0]//block_height
blocks_in_x = img.shape[1]//block_width
print(f"{blocks_in_y}x{blocks_in_x} blocks (vectors) in the image")
number_of_blocks = blocks_in_y*blocks_in_x
total_number_of_bits = number_of_blocks*bits_per_block
print(f"total number of output bytes = {total_number_of_bits//8}")

## RD performance

In [ ]:
def save(img, fn):
    io.imsave(fn, img, check_contrast=False)
    #subprocess.run(f"optipng {fn}", shell=True, capture_output=True)
    required_bytes = os.path.getsize(fn)
    print(f"Written {required_bytes} bytes in {fn}")
    return required_bytes

In [ ]:
def RD_curve(img, range_of_N_bins):
    blocks = []
    for i in range(0, img.shape[0], block_width):
        for j in range(0, img.shape[1], block_height):
            blocks.append(np.reshape(img[i:i + block_width, j:j + block_height], block_length))
    blocks = np.asarray(blocks).astype(float)
    points = []
    for n in range_of_N_bins:
        k_means = KMeans(init="k-means++", n_clusters=n, n_init=N_tries)
        k_means.fit(blocks)
        centroids = k_means.cluster_centers_.squeeze().astype(np.uint8)
        k = k_means.labels_.astype(np.uint8)  # bit-depth depends on number of bins! 
        k = k.reshape(img.shape[0]//block_height, img.shape[1]//block_width)
        y = np.empty_like(img)
        for i in range(0, img.shape[0], block_width):
            for j in range(0, img.shape[1], block_height):
                y[i:i + block_width, j:j + block_height] = centroids[k[i//block_width,j//block_height]].reshape(block_height, block_width)
        print("Quantization indexes: ", np.unique(k))
        #rate = gray_image.write(k, "/tmp/" + str(n) + '_', 0)*8/img.size
        rate = save(k, "/tmp/" + str(n) + ".png")*8/(k.shape[0]*k.shape[1])
        distortion = RMSE(img, y)
        #_distortion = distortion.RMSE(img, y)
        if not n%10:
            plt.title(f"{n}")
            plt.imshow(y, cmap=plt.cm.gray, vmin=0, vmax=256)
            plt.show()
        points.append((rate, distortion))
        print(f"n={n:>3}, rate={rate:>7} bits/pixel, distortion={distortion:>6.1f}")
    return points

In [ ]:
RD_points = RD_curve(img, range_of_N_bins)

In [ ]:
pylab.figure(dpi=150)
#pylab.scatter(*zip(*RD_points), label=f"VQ+PNG", s=1, marker='.')
pylab.plot(*zip(*RD_points), c='m', marker='x', label=f"VQ(2D)+PNG", linestyle="dotted")
pylab.title(f"Rate/Distortion Performance")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
pylab.show()

In [ ]:
with open(f"VQ_2D_RD_points.txt", 'w') as f:
    for item in RD_points:
        f.write(f"{item[0]}\t{item[1]}\n")

In [ ]:
input()